In [1]:
#Importing Dependencies
import numpy as np
from scipy.integrate import odeint
from matplotlib import pyplot as plt
from ipywidgets import interactive


%matplotlib inline

In [2]:
ierror = 0
prev_error = 0
#PID Module that takes input: y(Model Output),SP(Set Point), params(Tuning parameters) and delta_t(Time Interval)
#and returns: x(Model Input)
def PID(v, SP, params, delta_t):
    global ierror
    global prev_error
    error = SP - v
    ierror = ierror + error*delta_t
    derror = (error - prev_error)/delta_t
    
    #kp,ki,kd = get_parameters()
    kp=params["kp"]
    ki=params["ki"]
    kd=params["kd"]
    
    x = kp*error + ki*ierror + kd*derror
    prev_error = error
    return x, error

Equation
===

The system is governed by the following equation

$$ \ddot{y} + 2\dot{y} + y = (t - 0.5)x $$

In [3]:
def model(y_cap, t, params):
    """
        Given a vector <y, y1> returns <y1, y2> according to equation
        above.
    """
    y, y1 = y_cap # y, dy/dt
    x = params

    y2 = -2*y1 - y + (t - 0.5)*x
    dy_cap = y1, y2

    return dy_cap

In [4]:
def get_normalized_values():
    '''
        Returns normalized values of Kp, Kd and alpha according to some Logic, currently it returns constant values.
    '''
    kp_ = 0.5
    kd_ = 0.5
    alpha = 2
    return kp_, kd_, alpha

def get_pid_parameters():
    '''
        Returns actual Kp, Ki and Kd values by using normalized values of Kp,Kd and alpha in
        the euations given in Research paper. Currently it returns constant values.
    '''
    ku = 4.7
    tu = 3.3
    kpmax = 0.6*ku
    kpmin = 0.32*ku
    kdmax = 0.15*ku*tu
    kdmin = 0.08*ku*tu
    kp_, kd_, alpha = get_normalized_values()
    kp = (kpmax - kpmin)*kp_ + kpmin
    kd = (kdmax - kdmin)*kd_ + kdmin
    ki = kp**2/(alpha*kd)
    params = {'kp':kp, 'ki':ki, 'kd':kd}
    return params


In [11]:
#Here we Simulate out Model+PID controller over time-step of 0.1 Seconds
def simulation(kp=2.8,ki=1.7,kd=1.12):
    #PID Tuning parameters
    params = {'kp':kp,'ki':ki,'kd':kd}
    
    #Time-Span with time-step of 0.1 Seconds
    t = np.linspace(0, 14, 140)
    delta_t = t[1] - t[0]
    
    #Initialising y0
    y0 = 0
    #Storing y(t) responses
    y = np.ones(len(t))*y0
    #Storing y'(t) reponses
    y1 = np.ones(len(t))*y0
    
    #Storing x(t) responses
    x = np.zeros(len(t))
    
    #Storing e(t) responses
    e = np.zeros(len(t))

    #Step-Adding through set points
    SP = np.ones(len(t))
    
    #Simulating the PID+Model system over time-step of 0.1s
    for i in range(len(t)-1):
        #Here we can also call a function to input PID parameters at each time-step
        #params = get_pid_parameters()
        x[i+1], e[i+1] = PID(y[i], SP[i], params, delta_t)
        #Using odeint we are returned two values y(t) and y'(t) we store them in y, y1 respectively
        z = odeint(model, [y[i], y1[i]], t[i:i+2], args=(x[i+1],))
        y1[i+1] = z[-1][1]
        y[i+1] = z[-1][0]
    
    #Plotting
    plt.figure(figsize=(7, 7), dpi= 80, facecolor='w', edgecolor='k')
    plt.subplot(2,1,1)
    plt.plot(t, y, 'r-', label='y(t)')
    plt.plot(t, SP, 'b--', label='SP')
    plt.legend(loc='best')
    plt.subplot(2,1,2)
    plt.plot(t, e, 'k', label='e(t)')
    plt.ylabel('Error')
    plt.xlabel('Time')
    plt.legend(loc='best')


In [12]:
interactive_plt = interactive(simulation, kp=(-20.0,200.0), ki=(-20.0,20.0), kd=(-20.0,20.0))
interactive_plt

interactive(children=(FloatSlider(value=2.8, description='kp', max=200.0, min=-20.0), FloatSlider(value=1.7, d…